# Crypto scrap

## Aim of the project
Get a quick overview of any coin important data (source : CoinGecko)

## 1. Retrieve CoinGecko's data

In [27]:
import pandas as pd

from datetime import datetime
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import NoSuchElementException 

In [57]:
# Function to extract the % of good sentiment in the sentence scraped
# Input : string
# Output : int

def extract_good_sentiment_pct(sentence):
    res = []
    for char in sentence:
        if char.isdigit():
            res.append(char)
    n_char = ''
    for i in range(len(res)):
        n_char += res[i]
    n = int(n_char)
    return n/100

In [3]:
# Function to clean the price output of coingecko
# Input : unformated price (string)
# Output : price (int)

def format_price(price_string):
    price_string = price_string.replace("\u202f","")
    price_string = price_string.replace(" $US","")
    price_string = price_string.replace(",", ".")
    price = float(price_string)
    return price

In [4]:
def format_pct(pct_string):
    return(round(float(pct_string.replace('%',''))/100,3))

In [45]:
def click_sentiment_button(driver, xpath):
    try:
        driver.execute_script("window.scrollTo(0,2000)")
        wait = WebDriverWait(driver, 5)
        system = wait.until(ec.element_to_be_clickable((By.XPATH, xpath)))
        system.click()
    except:
        pass

In [58]:
# Function that returns coin gecko's data about a given coin
# Input : full coin name (ex: bitcoin, ethereum...) (string)
# Output : Number of favorites (int), current price in $ (float), 

def retrieve_coingecko_data(coin_name):
    
    driver = webdriver.Chrome('/Users/manuel/Documents/GitHub/chromedriver')
    driver.get(f"https://www.coingecko.com/fr/pi%C3%A8ces/{coin_name}")
    driver.implicitly_wait(10)
    
    price_string = driver.find_element_by_xpath('/html/body/div[4]/div[4]/div[1]/div/div[1]/div[4]/div/div[1]/span[1]/span').text
    price = format_price(price_string)
    
    hourly_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/span').text
    hourly_evolution = format_pct(hourly_evolution_string)
    
    daily_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[2]/span').text
    daily_evolution = format_pct(daily_evolution_string)
    
    weekly_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[3]/span').text
    weekly_evolution = format_pct(weekly_evolution_string)
    
    bimonthly_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[4]/span').text
    bimonthly_evolution = format_pct(bimonthly_evolution_string)
    
    monthly_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[5]/span').text
    monthly_evolution = format_pct(monthly_evolution_string)
    
    yearly_evolution_string = driver.find_element_by_xpath('/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[6]/span').text
    yearly_evolution = format_pct(yearly_evolution_string)
    
    market_cap_string = driver.find_element_by_xpath('/html/body/div[4]/div[4]/div[1]/div/div[2]/div[2]/div[1]/div/div/div[1]/span[2]/span').text
    market_cap = format_price(market_cap_string)
    
    daily_trading_string = driver.find_element_by_xpath('/html/body/div[4]/div[4]/div[1]/div/div[2]/div[2]/div[1]/div/div/div[2]/span[2]/span').text
    daily_trading = format_price(daily_trading_string)
    
    sentiment_xpath = "//*[@id='general']/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/a[1]"
    click_sentiment_button(driver, sentiment_xpath)
    good_sentiment = driver.find_element_by_xpath("/html/body/div[4]/div[6]/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[1]").get_attribute("style")
    good_sentiment_n = extract_good_sentiment_pct(good_sentiment)
    
    return ({"price ($US)":price,
             "price_evolution":{"1h":hourly_evolution, 
                                "24h":daily_evolution,
                                "7d": weekly_evolution,
                                "14d": bimonthly_evolution,
                                "1m": monthly_evolution,
                                "1y": yearly_evolution},
             "market cap":market_cap,
             "daily trading volume":daily_trading,
             "good_sentiment": good_sentiment_n})
    

In [60]:
coin="solana"
retrieve_coingecko_data(coin)

{'price ($US)': 185.9,
 'price_evolution': {'1h': -0.007,
  '24h': 0.029,
  '7d': 0.27,
  '14d': 1.113,
  '1m': 3.447,
  '1y': 52.835},
 'market cap': 54569328405.0,
 'daily trading volume': 5906181784.0,
 'good_sentiment': 0.76}